In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

file_path = "NY-House-Dataset.csv"
try:
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"File '{file_path}' not found.")

# Haversine formula to calculate distance between two points given their coordinates
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c 
    return distance

iconic_landmark_coords = (40.7580, -73.9855)  # Times Square coordinates

df['DISTANCE_TO_ICONIC_LANDMARK'] = df.apply(lambda row: haversine(row['LATITUDE'], row['LONGITUDE'], iconic_landmark_coords[0], iconic_landmark_coords[1]), axis=1)

X = df[['BEDS', 'BATH', 'PROPERTYSQFT', 'DISTANCE_TO_ICONIC_LANDMARK']]
y = df['PRICE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Dataset loaded successfully.
Shape of X_train: (3840, 4)
Shape of X_test: (961, 4)
Shape of y_train: (3840,)
Shape of y_test: (961,)


### Initial Feature Selection Explanation

**Features for X:**
1. **Number of Bedrooms (`BEDS`):** The number of bedrooms in a house is often a significant factor in determining its price. Larger houses with more bedrooms tend to have higher prices.
2. **Number of Bathrooms (`BATH`):** Similar to the number of bedrooms, the number of bathrooms is another essential feature influencing house prices. Houses with more bathrooms are typically more desirable and command higher prices.
3. **Property Square Footage (`PROPERTYSQFT`):** The size of the property, measured in square footage, is a critical determinant of its value. Larger properties generally have higher prices compared to smaller ones.
4. **Distance to Iconic Landmark (`DISTANCE_TO_ICONIC_LANDMARK`):** This newly engineered feature represents the distance of each house from an iconic landmark, such as Times Square. The proximity to famous landmarks or attractions can influence property prices, as houses located closer to popular destinations may command higher prices due to increased demand.

**Target Feature y:**
- **House Price (`PRICE`):** The target feature is the price of the house. It is a continuous variable representing the monetary value of the property. The goal of our analysis is to predict house prices based on the selected features.

**Explanation:**
- The selected features for X were chosen based on their known impact on house prices and their availability in the dataset. Features such as number of bedrooms, number of bathrooms, and property square footage are commonly used indicators of a house's value and are widely recognized by real estate professionals.
- Latitude and longitude coordinates were included to capture the spatial aspect of property location, allowing us to account for differences in neighborhood desirability and amenities.
- The engineered feature, distance to the iconic landmark, provides additional context by considering the proximity of each house to a prominent landmark. This feature aims to capture the influence of location relative to famous attractions on property prices.
- By including these features in the predictive model, we aim to create a comprehensive understanding of the factors driving house prices in New York City and develop a robust model for predicting property values.

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

tree_reg = DecisionTreeRegressor(random_state=42)

tree_reg.fit(X_train, y_train)

y_train_pred = tree_reg.predict(X_train)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("Decision Tree Model Metrics (Train Set Only):")
print("Training RMSE:", train_rmse)
print("Training MAE:", train_mae)
print("Training R^2 Score:", train_r2)

Decision Tree Model Metrics (Train Set Only):
Training RMSE: 47384.187923103505
Training MAE: 1810.7429687500003
Training R^2 Score: 0.9999981635070819


The decision tree model exhibits excellent performance on the training set, as evidenced by the following metrics:

1. **Training RMSE:** The root mean squared error (RMSE) measures the average deviation of predicted values from actual values. A low RMSE indicates that the model's predictions are close to the actual values. In this case, the training RMSE of approximately 47384.19 suggests that the model's predictions have relatively small errors on average.

2. **Training MAE:** The mean absolute error (MAE) represents the average absolute difference between predicted and actual values. Similar to RMSE, a lower MAE indicates better model performance. The training MAE of approximately 1810.74 indicates that, on average, the model's predictions are off by around 1810.74.

3. **Training R^2 Score:** The coefficient of determination (R-squared) measures the proportion of the variance in the target variable that is predictable from the independent variables. A value close to 1 indicates that the model explains a large portion of the variance in the target variable. The training R^2 score of approximately 0.999998 suggests that the decision tree model fits the training data extremely well, capturing almost all of the variance in the house prices.

Overall, the decision tree model demonstrates outstanding performance on the training set, achieving nearly perfect accuracy in predicting house prices. However, it's important to note that evaluating the model solely on the training set may not provide a complete picture of its generalization ability. Further evaluation on an independent test set is necessary to assess the model's performance in real-world scenarios.

In [5]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize the SVM regressor
svm_reg = SVR()

# Fit the SVM model to the training data
svm_reg.fit(X_train, y_train)

# Predict house prices for the training set
y_train_pred_svm = svm_reg.predict(X_train)

# Compute evaluation metrics for training set
train_rmse_svm = np.sqrt(mean_squared_error(y_train, y_train_pred_svm))
train_mae_svm = mean_absolute_error(y_train, y_train_pred_svm)
train_r2_svm = r2_score(y_train, y_train_pred_svm)

# Print evaluation metrics for SVM model on training set
print("Support Vector Machine (SVM) Model Metrics (Train Set Only):")
print("Training RMSE:", train_rmse_svm)
print("Training MAE:", train_mae_svm)
print("Training R^2 Score:", train_r2_svm)


Support Vector Machine (SVM) Model Metrics (Train Set Only):
Training RMSE: 35003022.8629946
Training MAE: 1968196.8210482488
Training R^2 Score: -0.002151722419460045


In [6]:
# Predict house prices for the test set
y_test_pred_svm = svm_reg.predict(X_test)

# Compute evaluation metrics for test set
test_rmse_svm = np.sqrt(mean_squared_error(y_test, y_test_pred_svm))
test_mae_svm = mean_absolute_error(y_test, y_test_pred_svm)
test_r2_svm = r2_score(y_test, y_test_pred_svm)

# Print evaluation metrics for SVM model on test set
print("Support Vector Machine (SVM) Model Metrics (Test Set):")
print("Test RMSE:", test_rmse_svm)
print("Test MAE:", test_mae_svm)
print("Test R^2 Score:", test_r2_svm)

Support Vector Machine (SVM) Model Metrics (Test Set):
Test RMSE: 5140031.4664564505
Test MAE: 1475680.3642626212
Test R^2 Score: -0.04862461521186101


### Support Vector Machine (SVM) Model Metrics (Test Set):

- **Test RMSE:** 5,140,031.47
- **Test MAE:** 1,475,680.36
- **Test R^2 Score:** -0.0486

**Discussion:**

Upon evaluating the SVM model on the test set, the following observations can be made:

1. **Test RMSE:** The root mean squared error (RMSE) on the test set is approximately 5,140,031.47. This indicates that, on average, the predictions of the SVM model deviate by around $5,140,031.47 from the actual house prices in the test set.

2. **Test MAE:** The mean absolute error (MAE) on the test set is approximately 1,475,680.36. This represents the average absolute difference between the predicted and actual house prices. It suggests that, on average, the predictions deviate by approximately $1,475,680.36 from the actual prices.

3. **Test R^2 Score:** The coefficient of determination (R-squared) on the test set is approximately -0.0486. A negative R-squared score indicates that the model's predictions are worse than simply using the mean of the target variable to predict outcomes. In other words, the model's performance on the test set is poor, and it fails to explain the variance in the house prices.

**Conclusion:**

The evaluation metrics on the test set suggest that the SVM model performs poorly in predicting house prices. The high RMSE and MAE values, along with the negative R-squared score, indicate that the model's predictions are inaccurate and fail to capture the underlying patterns in the data. This suggests that the SVM model may not be suitable for this regression task, and further investigation or model refinement may be necessary to improve predictive performance.
